### Train model với kernel_size = 1

### 1. Import Thư viện
- `json`, `os`: Xử lý file và thư mục
- `librosa`: Xử lý tín hiệu âm thanh
- `numpy`: Tính toán ma trận
- `tensorflow`: Framework deep learning

In [ ]:
import json
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

### 2. Định nghĩa Đường dẫn và Tải Metadata
- Định nghĩa đường dẫn đến các thư mục dữ liệu: train, test, validation
- Hàm `load_metadata`: Tải thông tin từ file JSON chứa metadata

In [ ]:
data_dir = {
    'train': r'D:\Python\AI-Instrument-Classifier\datasets\nsynth-train',
    'test': r'D:\Python\AI-Instrument-Classifier\datasets\nsynth-test',
    'valid': r'D:\Python\AI-Instrument-Classifier\datasets\nsynth-valid'
}

def load_metadata(split='train'):
    json_path = os.path.join(data_dir[split], 'examples.json')
    if not os.path.isfile(json_path):
        raise FileNotFoundError(f"Không tìm thấy file tại: {json_path}")
    with open(json_path, 'r') as f:
        metadata = json.load(f)
    return metadata

### 3. Tiền xử lý Âm thanh
- Hàm `preprocess_audio`: 
  - Chuẩn hóa độ dài audio
  - Chuyển đổi thành Mel Spectrogram
  - Chuyển sang đơn vị dB

In [ ]:
def preprocess_audio(audio, max_length=16000, n_mels=128, hop_length=512):
    if len(audio) > max_length:
        audio = audio[:max_length]
    else:
        audio = np.pad(audio, (0, max_length - len(audio)), 'constant')
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=16000, n_mels=n_mels, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

### 4. Generator Dữ liệu
- Hàm `data_generator`: 
  - Tạo generator để sinh dữ liệu theo batch
  - Trả về Mel Spectrogram và nhãn (instrument family)

In [ ]:
# Generator dữ liệu
def data_generator(metadata, split='train'):
    for note_str, info in metadata.items():
        audio_path = os.path.join(data_dir[split], 'audio', f'{note_str}.wav')
        audio, sr = librosa.load(audio_path, sr=16000)
        mel_spec = preprocess_audio(audio)
        label = info['instrument_family']
        yield mel_spec[..., np.newaxis], label

### 5. Chuẩn bị Dataset
- Tạo `tf.data.Dataset` cho train và validation
- Batch size: 64
- Sử dụng `prefetch` để tối ưu hiệu suất

In [ ]:
train_metadata = load_metadata('train')
valid_metadata = load_metadata('valid')

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_metadata, 'train'),
    output_types=(tf.float32, tf.int32),
    output_shapes=([128, 32, 1], [])
).batch(64).repeat().prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(valid_metadata, 'valid'),
    output_types=(tf.float32, tf.int32),
    output_shapes=([128, 32, 1], [])
).batch(64).repeat().prefetch(tf.data.AUTOTUNE)

### 6. Xây dựng Mô hình ResNet
- Hàm `residual_block`: Khối residual cơ bản
- Hàm `build_resnet_small`: 
  - Input: Mel Spectrogram (128x32x1)
  - Output: 11 lớp (số loại nhạc cụ)
  - Sử dụng L2 regularization và Dropout để giảm overfitting

In [ ]:
def residual_block(x, filters, kernel_size=3):
    y = layers.Conv2D(filters, kernel_size, padding='same', activation='relu',
                      kernel_regularizer=tf.keras.regularizers.l2(0.002))(x)
    y = layers.Conv2D(filters, kernel_size, padding='same',
                      kernel_regularizer=tf.keras.regularizers.l2(0.002))(y)
    shortcut = layers.Conv2D(filters, 1, padding='same')(x) if x.shape[-1] != filters else x
    y = layers.Add()([shortcut, y])
    y = layers.Activation('relu')(y)
    return y

def build_resnet_small(input_shape=(128, 32, 1), num_classes=11):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(16, (1, 1), padding='same', activation='relu',
                      kernel_regularizer=tf.keras.regularizers.l2(0.002))(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = residual_block(x, 16, kernel_size=1)   # Thay đổi kernel_size=1
    x = residual_block(x, 32, kernel_size=1)
    x = layers.MaxPooling2D((2, 2))(x)
    x = residual_block(x, 64, kernel_size=1)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu',
                     kernel_regularizer=tf.keras.regularizers.l2(0.002))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return models.Model(inputs, outputs)

### 7. Huấn luyện và Lưu Mô hình
- Compile với:
  - Optimizer: Adam (lr=0.003)
  - Loss: sparse_categorical_crossentropy
  - Metric: accuracy
- Callback:
  - EarlyStopping: Dừng sớm nếu val_loss không cải thiện sau 7 epochs
  - ReduceLROnPlateau: Giảm learning rate khi val_loss không cải thiện
- Lưu mô hình vào file `.h5`

In [ ]:
# Khởi tạo mô hình
model = build_resnet_small()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.003)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Tính số bước
train_steps = len(train_metadata) // 64
valid_steps = len(valid_metadata) // 64

# Huấn luyện
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=9,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)
    ]
)

# Đường dẫn lưu mô hình
model_save_path = r'D:\Python\AI-Instrument-Classifier\models\resnet_mel_instrument_classifier.h5'
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
model.save(model_save_path)
print(f"Đã huấn luyện xong và lưu mô hình tại: {model_save_path}")